In [1]:
import os
import gc
import pickle
import argparse
import datetime
import time
import json
import distutils.util
import pprint

import numpy as np
import tensorflow as tf
import scipy.constants
import sklearn

import data_pre_processing
import self_har_models
import self_har_utilities
import self_har_trainers
import transformations

__author__ = "C. I. Tang"
__copyright__ = "Copyright (C) 2021 C. I. Tang"

"""
Complementing the work of Tang et al.: SelfHAR: Improving Human Activity Recognition through Self-training with Unlabeled Data
@article{10.1145/3448112,
  author = {Tang, Chi Ian and Perez-Pozuelo, Ignacio and Spathis, Dimitris and Brage, Soren and Wareham, Nick and Mascolo, Cecilia},
  title = {SelfHAR: Improving Human Activity Recognition through Self-Training with Unlabeled Data},
  year = {2021},
  issue_date = {March 2021},
  publisher = {Association for Computing Machinery},
  address = {New York, NY, USA},
  volume = {5},
  number = {1},
  url = {https://doi.org/10.1145/3448112},
  doi = {10.1145/3448112},
  abstract = {Machine learning and deep learning have shown great promise in mobile sensing applications, including Human Activity Recognition. However, the performance of such models in real-world settings largely depends on the availability of large datasets that captures diverse behaviors. Recently, studies in computer vision and natural language processing have shown that leveraging massive amounts of unlabeled data enables performance on par with state-of-the-art supervised models.In this work, we present SelfHAR, a semi-supervised model that effectively learns to leverage unlabeled mobile sensing datasets to complement small labeled datasets. Our approach combines teacher-student self-training, which distills the knowledge of unlabeled and labeled datasets while allowing for data augmentation, and multi-task self-supervision, which learns robust signal-level representations by predicting distorted versions of the input.We evaluated SelfHAR on various HAR datasets and showed state-of-the-art performance over supervised and previous semi-supervised approaches, with up to 12% increase in F1 score using the same number of model parameters at inference. Furthermore, SelfHAR is data-efficient, reaching similar performance using up to 10 times less labeled data compared to supervised approaches. Our work not only achieves state-of-the-art performance in a diverse set of HAR datasets, but also sheds light on how pre-training tasks may affect downstream performance.},
  journal = {Proc. ACM Interact. Mob. Wearable Ubiquitous Technol.},
  month = mar,
  articleno = {36},
  numpages = {30},
  keywords = {semi-supervised training, human activity recognition, unlabeled data, self-supervised training, self-training, deep learning}
}

Access to Article:
    https://doi.org/10.1145/3448112
    https://dl.acm.org/doi/abs/10.1145/3448112

Contact: cit27@cl.cam.ac.uk

Copyright (C) 2021 C. I. Tang

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program. If not, see <https://www.gnu.org/licenses/>.

"""

LOGS_SUB_DIRECTORY = 'logs'
MODELS_SUB_DIRECTORY = 'models'


def get_parser():
    def strtobool(v):
        return bool(distutils.util.strtobool(v))


    parser = argparse.ArgumentParser(
        description='SelfHAR Training')
        
    parser.add_argument('--working_directory', default='run',
                        help='directory containing datasets, trained models and training logs')
    parser.add_argument('--config', default='sample_configs/self_har.json',
                        help='')
    
    parser.add_argument('--labelled_dataset_path', default='run/processed_datasets/motionsense_processed.pkl', type=str, 
                        help='name of the labelled dataset for training and fine-tuning')
    parser.add_argument('--unlabelled_dataset_path', default='run/processed_datasets/hhar_processed.pkl', type=str, 
                        help='name of the unlabelled dataset to self-training and self-supervised training, ignored if only supervised training is performed.')
    
    parser.add_argument('--window_size', default=400, type=int,
                        help='the size of the sliding window')
    parser.add_argument('--max_unlabelled_windows', default=40000, type=int,
                        help='')

    parser.add_argument('--use_tensor_board_logging', default=True, type=strtobool,
                        help='')
    parser.add_argument('--verbose', default=1, type=int,
                        help='verbosity level')

    return parser

def prepare_dataset(dataset_path, window_size, get_train_test_users, validation_split_proportion=0.1, verbose=1):
    if verbose > 0:
        print(f"Loading dataset at {dataset_path}")

    with open(dataset_path, 'rb') as f:
        dataset_dict = pickle.load(f)
        user_datasets = dataset_dict['user_split']
        label_list = dataset_dict['label_list'] # ['sit', 'std', 'wlk', 'ups', 'dws', 'jog']

    label_map = dict([(l, i) for i, l in enumerate(label_list)]) # {'sit': 0, 'std': 1, 'wlk': 2, 'ups': 3, 'dws': 4, 'jog': 5}
    output_shape = len(label_list) # 6

    har_users = list(user_datasets.keys())
    train_users, test_users = get_train_test_users(har_users)
    if verbose > 0:
        print(f'Testing users: {test_users}, Training users: {train_users}')
        # Testing users: [1, 14, 19, 23, 6], Training users: [10, 11, 12, 13, 15, 16, 17, 18, 2, 20, 21, 22, 24, 3, 4, 5, 7, 8, 9]

    np_train, np_val, np_test = data_pre_processing.pre_process_dataset_composite(
        user_datasets=user_datasets, 
        label_map=label_map, 
        output_shape=output_shape, 
        train_users=train_users, 
        test_users=test_users, 
        window_size=window_size, 
        shift=window_size//2, 
        normalise_dataset=True, 
        validation_split_proportion=validation_split_proportion,
        verbose=verbose
    )

    return {
        'train': np_train,
        'val': np_val,
        'test': np_test,
        'label_map': label_map,
        'input_shape': np_train[0].shape[1:],
        'output_shape': output_shape,
    }

def generate_unlabelled_datasets_variations(unlabelled_data_x, labelled_data_x, labelled_repeat=1, verbose=1):
    if verbose > 0:
        print("Unlabeled data shape: ", unlabelled_data_x.shape) # (56383, 400, 3)
    
    labelled_data_repeat = np.repeat(labelled_data_x, labelled_repeat, axis=0) # (4689, 400, 3)
    np_unlabelled_combined = np.concatenate([unlabelled_data_x, labelled_data_repeat]) # (61072, 400, 3)
    if verbose > 0:
        print(f"Unlabelled Combined shape: {np_unlabelled_combined.shape}")
    gc.collect()

    return {
        'labelled_x_repeat': labelled_data_repeat,
        'unlabelled_combined': np_unlabelled_combined
    }

def load_unlabelled_dataset(prepared_datasets, unlabelled_dataset_path, window_size, labelled_repeat, max_unlabelled_windows=None, verbose=1):
    def get_empty_test_users(har_users):
        return (har_users, [])

    prepared_datasets['unlabelled'] = prepare_dataset(unlabelled_dataset_path, window_size, get_empty_test_users, validation_split_proportion=0, verbose=verbose)['train'][0]
    if max_unlabelled_windows is not None:
        prepared_datasets['unlabelled'] = prepared_datasets['unlabelled'][:max_unlabelled_windows]
    prepared_datasets = {
        **prepared_datasets,
        **generate_unlabelled_datasets_variations(
            prepared_datasets['unlabelled'], 
            prepared_datasets['labelled']['train'][0],
            labelled_repeat=labelled_repeat
    )}
    return prepared_datasets

def get_config_default_value_if_none(experiment_config, entry, set_value=True):
    if entry in experiment_config:
        return experiment_config[entry]
    
    if entry == 'type':
        default_value = 'none'
    elif entry == 'tag':
        default_value = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    elif entry == 'previous_config_offset':
        default_value = 0
    elif entry == 'initial_learning_rate':
        default_value = 0.0003
    elif entry == 'epochs':
        default_value = 30
    elif entry == 'batch_size':
        default_value = 300
    elif entry == 'optimizer':
        default_value = 'adam'
    elif entry == 'self_training_samples_per_class':
        default_value = 10000
    elif entry == 'self_training_minimum_confidence':
        default_value = 0.0
    elif entry == 'self_training_plurality_only':
        default_value = True
    elif entry == 'trained_model_path':
        default_value = ''
    elif entry == 'trained_model_type':
        default_value = 'unknown'
    elif entry == 'eval_results':
        default_value = {}
    elif entry == 'eval_har':
        default_value = False

    if set_value:
        experiment_config[entry] = default_value
        print(f"INFO: configuration {entry} set to default value: {default_value}.")
    
    return default_value

/home/shmiao/anaconda3/envs/tensorflow36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shmiao/anaconda3/envs/tensorflow36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shmiao/anaconda3/envs/tensorflow36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shmiao/anaconda3/envs/

In [2]:
parser = get_parser()
args = parser.parse_args(args=[])

current_time_string = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") # 20211207-172629
working_directory = args.working_directory # 'run'
verbose = args.verbose # 1
use_tensor_board_logging = args.use_tensor_board_logging # True
window_size = args.window_size # 400

if use_tensor_board_logging:
    logs_directory = os.path.join(working_directory, LOGS_SUB_DIRECTORY) # 'run/logs'
    if not os.path.exists(logs_directory):
        os.mkdir(logs_directory)
models_directory = os.path.join(working_directory, MODELS_SUB_DIRECTORY) # 'run/models'
if not os.path.exists(models_directory):
    os.mkdir(models_directory)
transform_funcs_vectorized = [
    transformations.noise_transform_vectorized, 
    transformations.scaling_transform_vectorized, 
    transformations.rotation_transform_vectorized, 
    transformations.negate_transform_vectorized, 
    transformations.time_flip_transform_vectorized, 
    transformations.time_segment_permutation_transform_improved, 
    transformations.time_warp_transform_low_cost, 
    transformations.channel_shuffle_transform_vectorized
]
transform_funcs_names = ['noised', 'scaled', 'rotated', 'negated', 'time_flipped', 'permuted', 'time_warped', 'channel_shuffled']

prepared_datasets = {}
labelled_repeat = 1             # TODO: improve flexibility transformation_multiple


def get_fixed_split_users(har_users):   # TODO: improve flexibility
    test_users = har_users[0::5]
    train_users = [u for u in har_users if u not in test_users]
    return (train_users, test_users)


prepared_datasets['labelled'] = prepare_dataset(args.labelled_dataset_path, window_size, get_fixed_split_users, validation_split_proportion=0.1, verbose=verbose)
input_shape = prepared_datasets['labelled']['input_shape']     #  (400, 3)
output_shape = prepared_datasets['labelled']['output_shape']   #   6


with open(args.config, 'r') as f:
    config_file = json.load(f)
    file_tag = config_file['tag'] # Self_HAR
    experiment_configs = config_file['experiment_configs']

if verbose > 0:
    print("Experiment Settings:")
    for i, config in enumerate(experiment_configs):
        print(f"Experiment {i}:")
        print(config)
        print("------------")
    time.sleep(5)



for i, experiment_config in enumerate(experiment_configs):
    if verbose > 0:
        print("---------------------")
        print(f"Starting Experiment {i}: {experiment_config}")
        print("---------------------")
        time.sleep(5)
    gc.collect()
    tf.keras.backend.clear_session()

Loading dataset at run/processed_datasets/motionsense_processed.pkl
Testing users: [1, 14, 19, 23, 6], Training users: [10, 11, 12, 13, 15, 16, 17, 18, 2, 20, 21, 22, 24, 3, 4, 5, 7, 8, 9]
Test
(array(['dws', 'jog', 'sit', 'std', 'ups', 'wlk'], dtype='<U3'), array([112, 133, 360, 335, 148, 331]))
(array([0, 1, 2, 3, 4, 5]), array([360, 335, 331, 148, 112, 133]))
-----------------
Train
(array(['dws', 'jog', 'sit', 'std', 'ups', 'wlk'], dtype='<U3'), array([ 449,  480, 1282, 1146,  546, 1308]))
(array([0, 1, 2, 3, 4, 5]), array([1282, 1146, 1308,  546,  449,  480]))
-----------------
Training data shape: (4689, 400, 3)
Validation data shape: (522, 400, 3)
Testing data shape: (1419, 400, 3)
Experiment Settings:
Experiment 0:
{'type': 'har_full_train', 'tag': 'Teacher_Train', 'previous_config_offset': 0, 'optimizer': 'adam', 'initial_learning_rate': 0.0003, 'epochs': 30, 'batch_size': 300}
------------
Experiment 1:
{'type': 'self_har', 'tag': 'Self_HAR_Pre_Train', 'previous_config_offset

In [4]:
for i, experiment_config in enumerate(experiment_configs):
    print(i, experiment_config)
    if i == 1:
        break

0 {'type': 'har_full_train', 'tag': 'Teacher_Train', 'previous_config_offset': 0, 'optimizer': 'adam', 'initial_learning_rate': 0.0003, 'epochs': 30, 'batch_size': 300}
1 {'type': 'self_har', 'tag': 'Self_HAR_Pre_Train', 'previous_config_offset': 1, 'optimizer': 'adam', 'initial_learning_rate': 0.0003, 'epochs': 30, 'batch_size': 300, 'self_training_samples_per_class': 10000, 'self_training_minimum_confidence': 0.5, 'self_training_plurality_only': True}


In [5]:
experiment_type = get_config_default_value_if_none(experiment_config, 'type') # har_full_train
    
if get_config_default_value_if_none(experiment_config, 'previous_config_offset') == 0:
    previous_config = None
else:
    previous_config = experiment_configs[i - experiment_config['previous_config_offset']]

if verbose > 0:
    print("Previous config: ", previous_config)

tag = f"{current_time_string}_{file_tag}_{get_config_default_value_if_none(experiment_config, 'tag')}"

Previous config:  {'type': 'har_full_train', 'tag': 'Teacher_Train', 'previous_config_offset': 0, 'optimizer': 'adam', 'initial_learning_rate': 0.0003, 'epochs': 30, 'batch_size': 300}


In [6]:
experiment_type

'self_har'

In [7]:
if 'unlabelled' not in prepared_datasets:
    prepared_datasets = load_unlabelled_dataset(prepared_datasets, args.unlabelled_dataset_path, window_size, labelled_repeat, max_unlabelled_windows=args.max_unlabelled_windows)

Loading dataset at run/processed_datasets/hhar_processed.pkl
Testing users: [], Training users: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']
Test
(array([], dtype=float64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
-----------------
Train
(array(['bike', 'sit', 'stairsdown', 'stairsup', 'stand', 'walk'],
      dtype=object), array([ 9215,  9957,  8080,  8912,  9258, 10961]))
(array([0, 1, 2, 3, 4, 5]), array([ 9957,  9258, 10961,  8912,  8080,  9215]))
-----------------
Training data shape: (56383, 400, 3)
Validation data shape: None
Testing data shape: (0, 0)
Unlabeled data shape:  (40000, 400, 3)
Unlabelled Combined shape: (44689, 400, 3)


In [9]:
prepared_datasets.keys()

dict_keys(['labelled', 'unlabelled', 'labelled_x_repeat', 'unlabelled_combined'])

In [10]:
for i, experiment_config in enumerate(experiment_configs):
    print(i, experiment_config)
    if i == 2:
        break

0 {'type': 'har_full_train', 'tag': 'Teacher_Train', 'previous_config_offset': 0, 'optimizer': 'adam', 'initial_learning_rate': 0.0003, 'epochs': 30, 'batch_size': 300}
1 {'type': 'self_har', 'tag': 'Self_HAR_Pre_Train', 'previous_config_offset': 1, 'optimizer': 'adam', 'initial_learning_rate': 0.0003, 'epochs': 30, 'batch_size': 300, 'self_training_samples_per_class': 10000, 'self_training_minimum_confidence': 0.5, 'self_training_plurality_only': True}
2 {'type': 'har_full_fine_tune', 'tag': 'Student_Fine_Tune', 'previous_config_offset': 1, 'optimizer': 'adam', 'initial_learning_rate': 0.0003, 'epochs': 30, 'batch_size': 300, 'eval_har': True}
